In [1]:
!pip install rapidfuzz


   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.6 MB 919.0 kB/s eta 0:00:02
   ------------ --------------------------- 0.5/1.6 MB 4.0 MB/s eta 0:00:01
   ------------------- -------------------- 0.8/1.6 MB 4.5 MB/s eta 0:00:01
   -------------------------------- ------- 1.3/1.6 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------  1.6/1.6 MB 6.8 MB/s eta 0:00:01
   ---------------------------------------- 1.6/1.6 MB 6.0 MB/s eta 0:00:00


In [2]:
from rapidfuzz import process as rf_process
from rapidfuzz import fuzz as rf
import pandas as pd


df1 = pd.read_csv('./data/전국연도별방문자추이.csv', encoding='euc-kr')
df2 = pd.read_csv('./data/회계정보전처리.csv', encoding='utf-8')

index_col1 = 'index'  
index_col2 = 'index'  

def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower().replace(" ", "")

df1['normalized_index'] = df1[index_col1].apply(normalize_text)
df2['normalized_index'] = df2[index_col2].apply(normalize_text)

similarities = []
for idx1, original_index in zip(df1['normalized_index'], df1[index_col1]):
    match = rf_process.extractOne(idx1, df2['normalized_index'], scorer=rf.ratio)
    similarities.append({
        'df1_original': original_index,
        'df1_normalized': idx1,
        'df2_best_match': match[0],
        'df2_original': df2[df2['normalized_index'] == match[0]][index_col2].values[0],
        'similarity_score': match[1]
    })

similarities_df = pd.DataFrame(similarities)

# 유사도 기준 필터링 (예: 80 이상)
filtered_df = similarities_df[similarities_df['similarity_score'] >= 80]

filtered_df.to_csv('./data/matched_similarities.csv', index=False, encoding='utf-8-sig')


KeyboardInterrupt: 

In [ ]:
matched_df = pd.read_csv('./data/matched_similarities.csv')

df2_matched = pd.merge(
    df1, 
    matched_df[['df1_original', 'df2_original']], 
    left_on='index',  
    right_on='df1_original',  
    how='left'  
)


final_df = pd.merge(
    df2_matched,  
    df2,  
    left_on='df2_original',  
    right_on='index', 
    how='left' 
)

print(final_df.head())

final_df.to_csv('./data/전국연도별방문자회계정보.csv', index=False, encoding='utf-8-sig')

      축제명  개최년도  축체기간(일)  (현지인)방문자수  (외지인)방문자수  (외국인)방문자수  (전체)방문자수  일평균 방문자수  \
0  강경젓갈축제  2018        5    50642.0    59291.0      108.0  110041.0  22008.20   
1  강경젓갈축제  2022        5    57971.0    69910.0       19.0  127900.0  25580.00   
2  강경젓갈축제  2023        4    52282.0    71999.0       18.0  124299.0  31074.75   
3  강릉커피축제  2018        5    61368.0    82621.0      279.0  144268.0  28853.60   
4  강릉커피축제  2019        4    98907.0    61246.0      743.0  160896.0  40224.00   

   전년도 일평균 방문자수  일평균 방문자수 증감률  ...    회계연도  지역명   행사축제명           총비용  \
0           NaN           NaN  ...  2018.0   충남  강경젓갈축제  8.250000e+08   
1       22008.2          16.2  ...  2022.0   충남  강경젓갈축제  1.003000e+09   
2       25580.0          21.5  ...  2023.0   충남  강경젓갈축제  1.178086e+09   
3           NaN           NaN  ...  2018.0   강원  강릉커피축제  3.904450e+08   
4       28853.6          39.4  ...  2019.0   강원  강릉커피축제  4.400000e+08   

          사업수익           순원가   시작월   종료월     index_y normalized_index_y  


In [ ]:
df.head()

,축제명,개최년도,축체기간(일),(현지인)방문자수,(외지인)방문자수,(외국인)방문자수,(전체)방문자수,일평균 방문자수,전년도 일평균 방문자수,일평균 방문자수 증감률,...,시군구,회계연도,지역명,행사축제명,총비용,사업수익,순원가,시작월,종료월,index
0,강경젓갈축제,2018,5,50642.0,59291.0,108.0,110041.0,22008.20,NaN,NaN,...,논산시,2018.0,충남,강경젓갈축제,8.250000e+08,183950000.0,6.410500e+08,10.0,10.0,강경젓갈축제2018
1,강경젓갈축제,2022,5,57971.0,69910.0,19.0,127900.0,25580.00,22008.2,16.2,...,논산시,2022.0,충남,강경젓갈축제,1.003000e+09,0.0,1.003000e+09,10.0,10.0,강경젓갈축제2022
2,강경젓갈축제,2023,4,52282.0,71999.0,18.0,124299.0,31074.75,25580.0,21.5,...,논산시,2023.0,충남,강경젓갈축제,1.178086e+09,0.0,1.178086e+09,10.0,10.0,강경젓갈축제2023
3,강릉커피축제,2018,5,61368.0,82621.0,279.0,144268.0,28853.60,NaN,NaN,...,강릉시,2018.0,강원,강릉커피축제,3.904450e+08,40000000.0,3.504450e+08,10.0,10.0,강릉커피축제2018
4,강릉커피축제,2019,4,98907.0,61246.0,743.0,160896.0,40224.00,28853.6,39.4,...,강릉시,2019.0,강원,강릉커피축제,4.400000e+08,91000000.0,3.490000e+08,10.0,10.0,강릉커피축제2019


In [ ]:
df = df.drop(['index_y', 'normalized_index_y','normalized_index_x', 'df1_original','df2_original','index_x'], axis = 1)

In [ ]:
df['index'] = df['축제명'] + df['개최년도'].astype(str)
df.to_csv('./data/전국연도별방문자회계정보진짜최종.csv', index=False, encoding='utf-8-sig')

In [ ]:
df.columns

Index(['축제명', '개최년도', '축체기간(일)', '(현지인)방문자수', '(외지인)방문자수', '(외국인)방문자수',
       '(전체)방문자수', '일평균 방문자수', '전년도 일평균 방문자수', '일평균 방문자수 증감률', '(이전)전체방문자',
       '(전체)방문자증감', '(현지인)방문자비율', '(외지인)방문자비율', '(외국인)방문자비율', '전년대비방문자증감비율',
       '도', '시군구', '회계연도', '지역명', '행사축제명', '총비용', '사업수익', '순원가', '시작월', '종료월',
       'index'],
      dtype='object')

In [ ]:
from rapidfuzz import process as rf_process
from rapidfuzz import fuzz
import pandas as pd

df1 = pd.read_csv('./전국연도별방문자추이.csv', encoding='euc-kr')
df2 = pd.read_csv('./회계정보전처리.csv', encoding='utf-8')

def normalize_text(text):
    if pd.isna(text):
        return ""
    return str(text).lower().replace(" ", "")

df1['normalized_index'] = df1['index'].apply(normalize_text)
df2['normalized_index'] = df2['index'].apply(normalize_text)

df1['개최년도'] = df1['개최년도'].astype(int)
df2['회계연도'] = df2['회계연도'].astype(int)

matches = []
for _, row1 in df1.iterrows():
    match = rf_process.extractOne(
        row1['normalized_index'],
        df2['normalized_index'],
        scorer=fuzz.ratio
    )
    if match and match[1] >= 75:  
        matched_row = df2[df2['normalized_index'] == match[0]]
        if not matched_row.empty:
            matched_row = matched_row.iloc[0]  
            if (
                row1['도'][:2] == matched_row['지역명'][:2] and  
                row1['개최년도'] == matched_row['회계연도'] 
            ):
                matches.append({
                    'df1_index': row1['index'],
                    'df2_index': matched_row['index'],
                    'similarity_score': match[1],
                    '도': row1['도'],
                    '개최년도': row1['개최년도']
                })

matches_df = pd.DataFrame(matches)

print(matches_df.head())

matches_df.to_csv('./matched_results_75.csv', index=False, encoding='utf-8-sig')


    df1_index   df2_index  similarity_score   도  개최년도
0  강경젓갈축제2018  강경젓갈축제2018             100.0  충남  2018
1  강경젓갈축제2022  강경젓갈축제2022             100.0  충남  2022
2  강경젓갈축제2023  강경젓갈축제2023             100.0  충남  2023
3  강릉커피축제2018  강릉커피축제2018             100.0  강원  2018
4  강릉커피축제2019  강릉커피축제2019             100.0  강원  2019


In [ ]:
matched_df = pd.read_csv('./matched_results_75.csv')

df2_matched = pd.merge(
    df1, 
    matched_df[['df1_index', 'df2_index']], 
    left_on='index',  
    right_on='df1_index',  
    how='left'  
)


final_df = pd.merge(
    df2_matched,  
    df2,  
    left_on='df2_index',  
    right_on='index', 
    how='left' 
)

print(final_df.head())

final_df.to_csv('./전국연도별방문자회계정보정망정말최종ver.csv', index=False, encoding='utf-8-sig')

      축제명  개최년도  축체기간(일)  (현지인)방문자수  (외지인)방문자수  (외국인)방문자수  (전체)방문자수  일평균 방문자수  \
0  강경젓갈축제  2018        5    50642.0    59291.0      108.0  110041.0  22008.20   
1  강경젓갈축제  2022        5    57971.0    69910.0       19.0  127900.0  25580.00   
2  강경젓갈축제  2023        4    52282.0    71999.0       18.0  124299.0  31074.75   
3  강릉커피축제  2018        5    61368.0    82621.0      279.0  144268.0  28853.60   
4  강릉커피축제  2019        4    98907.0    61246.0      743.0  160896.0  40224.00   

   전년도 일평균 방문자수  일평균 방문자수 증감률  ...    회계연도  지역명   행사축제명           총비용  \
0           NaN           NaN  ...  2018.0   충남  강경젓갈축제  8.250000e+08   
1       22008.2          16.2  ...  2022.0   충남  강경젓갈축제  1.003000e+09   
2       25580.0          21.5  ...  2023.0   충남  강경젓갈축제  1.178086e+09   
3           NaN           NaN  ...  2018.0   강원  강릉커피축제  3.904450e+08   
4       28853.6          39.4  ...  2019.0   강원  강릉커피축제  4.400000e+08   

          사업수익           순원가   시작월   종료월     index_y normalized_index_y  
